![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# 🎯 Context Types in Practice

## 📚 Quick Recap: What You've Learned

In the previous notebook, you discovered:

### The 4 Core Context Types
1. **System Context** 📌 - The AI's role and knowledge (static)
2. **User Context** 👤 - Personal profile and preferences (dynamic)
3. **Conversation Context** 💬 - Dialogue history (dynamic)
4. **Retrieved Context** 🔍 - Query-specific data (dynamic)

### Key Insights
- **Context window limits** everything you can include
- **Every token counts** - optimize for relevance
- **Static context** = universal, hardcoded, fast
- **Dynamic context** = personalized, retrieved, flexible

---

## 🎓 What You'll Learn (20-25 minutes)

Now let's put these concepts into practice:
1. 🔧 Build each context type step-by-step
2. 🎯 Combine contexts for intelligent responses
3. ⚡ Optimize context management strategies
4. 💻 Create production-ready patterns

Let's dive in!


## 🔧 Setup

Let's start with a simple setup - just the essentials. You will need to load your OpenAI Key

In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

---

## 1️⃣ System Context: The AI's Identity

System context defines **what the AI is** and **what it knows**.

### 📋 What Goes in System Context?
- 🎭 Role and personality
- 📚 Domain knowledge
- 📋 Business rules
- 🛠️ Available tools

### ✨ Characteristics
- ✅ Same for all users
- ✅ Rarely changes
- ✅ Hardcoded in your application

### 💻 Let's Build System Context Step-by-Step

In [3]:
# Step 1: Define the AI's role
system_context = """You are a Redis University course advisor."""

In [4]:
# Step 2: Add domain knowledge (available courses)
system_context = """You are a Redis University course advisor.

Available courses:
- RU101: Introduction to Redis (Beginner)
- RU201: Redis for Python (Intermediate, requires RU101)
- RU202: Redis for Java (Intermediate, requires RU101)
- RU301: Vector Similarity Search (Advanced, requires RU201 or RU202)
"""

In [6]:
# Step 3: Add behavioral instructions
system_context = """You are a Redis University course advisor.

Available courses:
- RU101: Introduction to Redis (Beginner)
- RU201: Redis for Python (Intermediate, requires RU101)
- RU202: Redis for Java (Intermediate, requires RU101)
- RU301: Vector Similarity Search (Advanced, requires RU201 or RU202)

Always provide specific recommendations with clear reasoning.
"""

# View the final system context
print(system_context)

You are a Redis University course advisor.

Available courses:
- RU101: Introduction to Redis (Beginner)
- RU201: Redis for Python (Intermediate, requires RU101)
- RU202: Redis for Java (Intermediate, requires RU101)
- RU301: Vector Similarity Search (Advanced, requires RU201 or RU202)

Always provide specific recommendations with clear reasoning.



💡 **Key Insight:** System context is the same for every user, every time. It's your AI's "personality" and "knowledge base."

---

## 2️⃣ User Context: Personal Information

User context contains **information about the specific user** that enables personalization.

### 📋 What Goes in User Context?
- 👤 User profile (name, background)
- ⭐ Preferences
- 📜 History (completed courses, past interactions)
- 🎯 Goals

### ✨ Characteristics
- ✅ Different for each user
- ✅ Retrieved from database
- ✅ Updates over time

### 💻 Let's Build User Context Step-by-Step

In [7]:
# Step 1: Create a simple user profile as a dictionary
sarah_profile = {
    "name": "Sarah Chen",
    "background": "Python developer, 2 years experience",
    "completed_courses": ["RU101"],
    "interests": ["machine learning", "data science", "python"]
}

sarah_profile

{'name': 'Sarah Chen',
 'background': 'Python developer, 2 years experience',
 'completed_courses': ['RU101'],
 'interests': ['machine learning', 'data science', 'python']}

In [8]:
# Step 2: Format it as context for the LLM
user_context = f"""Student Profile:
- Name: {sarah_profile['name']}
- Background: {sarah_profile['background']}
- Completed: {', '.join(sarah_profile['completed_courses'])}
- Interests: {', '.join(sarah_profile['interests'])}
"""

user_context

'Student Profile:\n- Name: Sarah Chen\n- Background: Python developer, 2 years experience\n- Completed: RU101\n- Interests: machine learning, data science, python\n'

### 🔄 Different Users = Different Context

Let's create another user to see how context changes:

In [12]:
# Create a different user profile
alex_profile = {
    "name": "Alex Kumar",
    "background": "Java backend engineer, 5 years experience",
    "completed_courses": ["RU101", "RU202"],
    "interests": ["distributed systems", "performance", "java"]
}

alex_context = f"""Student Profile:
- Name: {alex_profile['name']}
- Background: {alex_profile['background']}
- Completed: {', '.join(alex_profile['completed_courses'])}
- Interests: {', '.join(alex_profile['interests'])}
"""

print(alex_context)

Student Profile:
- Name: Alex Kumar
- Background: Java backend engineer, 5 years experience
- Completed: RU101, RU202
- Interests: distributed systems, performance, java



💡 **Key Insight:** Each user gets personalized context. In production, you'd fetch this from a database based on user ID.

---

## 3️⃣ Conversation Context: Dialogue History

Conversation context maintains **the flow of dialogue** and enables follow-up questions.

### 📋 What Goes in Conversation Context?
- 💬 Previous messages
- ❓ Questions asked
- 💡 Answers given
- 🔄 Current conversation flow

### ✨ Characteristics
- ✅ Session-specific
- ✅ Grows with each exchange
- ✅ Enables follow-up questions

### 💻 Let's Build Conversation Context Step-by-Step

In [ ]:
# Step 1: Start with an empty conversation
conversation = []

In [ ]:
# Step 2: Add the first user message
conversation.append({
    "role": "user",
    "content": "What Redis course should I take next?"
})

conversation

In [ ]:
# Step 3: Add the assistant's response
conversation.append({
    "role": "assistant",
    "content": "Based on your Python background, I recommend RU201 (Redis for Python)."
})

conversation

In [ ]:
# Step 4: Add a follow-up question
conversation.append({
    "role": "user",
    "content": "How long will that take?"
})

conversation

### 🎯 Why Conversation Context Matters

Notice the question **"How long will that take?"**

- ❌ Without conversation context: The AI doesn't know what "that" refers to
- ✅ With conversation context: The AI knows "that" = RU201 from the previous exchange

💡 **Key Insight:** Conversation context enables natural, flowing dialogue with pronouns and references.

---

## 4️⃣ Retrieved Context: Query-Specific Data

Retrieved context is **information fetched based on the current query**.

### 📋 What Goes in Retrieved Context?
- 🔍 Search results
- 💾 Database queries
- 🌐 API responses
- ⏱️ Real-time data

### ✨ Characteristics
- ✅ Query-specific
- ✅ Retrieved at runtime
- ✅ Most relevant to current need

> 💡 **Note:** We'll dive deep into this in Section 2 (RAG - Retrieval-Augmented Generation)!

### 💻 Let's Build Retrieved Context Step-by-Step

In [ ]:
# Step 1: Simulate a course database
course_database = {
    "RU201": {
        "title": "Redis for Python",
        "level": "Intermediate",
        "description": "Learn to use Redis with Python applications",
        "duration": "6-8 hours",
        "prerequisites": ["RU101"]
    },
    "RU301": {
        "title": "Vector Similarity Search",
        "level": "Advanced",
        "description": "Master vector search with Redis",
        "duration": "8-10 hours",
        "prerequisites": ["RU201", "RU202"]
    }
}

In [ ]:
# Step 2: Retrieve course info based on query
def get_course_info(course_code):
    """Simulate retrieving course information from database"""
    return course_database.get(course_code, {})

# Retrieve RU201 info
ru201_info = get_course_info("RU201")
ru201_info

In [ ]:
# Step 3: Format as context for the LLM
retrieved_context = f"""Course Information:
- Code: RU201
- Title: {ru201_info['title']}
- Level: {ru201_info['level']}
- Description: {ru201_info['description']}
- Duration: {ru201_info['duration']}
- Prerequisites: {', '.join(ru201_info['prerequisites'])}
"""

retrieved_context

💡 **Key Insight:** Retrieved context is fetched **on-demand** based on what the user is asking about. Different queries = different retrieved context.

---

## 🎯 Bringing It All Together

Now let's combine all 4 context types to create an intelligent LLM call!

### 💻 Step-by-Step Context Integration

In [ ]:
# Step 1: Build the messages array with all context types
messages = [
    # 1. System Context
    {"role": "system", "content": system_context},
    
    # 2. User Context
    {"role": "user", "content": user_context},
]

messages

In [ ]:
# Step 2: Add conversation history (if any)
if conversation:
    messages.extend(conversation)

messages

In [ ]:
# Step 3: Add retrieved context (if relevant)
messages.append({
    "role": "system",
    "content": retrieved_context
})

messages

In [ ]:
# Step 4: Add the current user query
messages.append({
    "role": "user",
    "content": "Should I take this course?"
})

messages

In [ ]:
# Step 5: Make the LLM call with complete context
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)

response.choices[0].message.content

### 🎯 What Just Happened?

The LLM received **all 4 context types**:

1. **System Context** 📌 - Knows it's a course advisor
2. **User Context** 👤 - Knows Sarah's background and interests
3. **Conversation Context** 💬 - Knows what was discussed
4. **Retrieved Context** 🔍 - Has detailed RU201 course info

Result: **Personalized, context-aware recommendation!** 🚀

---

## 📊 Context Management Strategies

Different scenarios require different approaches to context management.

### Strategy 1: New User (Minimal Context)

| Context Type | What to Include |
|--------------|----------------|
| 📌 System | Full role definition |
| 👤 User | Basic profile only |
| 💬 Conversation | Empty (new session) |
| 🔍 Retrieved | General information |

**Use when:** First-time user, no history available

### Strategy 2: Returning User (Rich Context)

| Context Type | What to Include |
|--------------|----------------|
| 📌 System | Full role definition |
| 👤 User | Complete profile + history |
| 💬 Conversation | Recent conversation history |
| 🔍 Retrieved | Personalized, relevant info |

**Use when:** User with history, ongoing conversation

### Strategy 3: Long Conversation (Optimized Context)

| Context Type | What to Include |
|--------------|----------------|
| 📌 System | Condensed role definition |
| 👤 User | Key profile elements only |
| 💬 Conversation | Summarized or recent only |
| 🔍 Retrieved | Highly relevant info only |

**Use when:** Approaching context window limits

---

## 🎉 Key Takeaways

Congratulations! You've mastered context types in practice:

### The 4 Context Types
1. **System Context** 📌 - AI's role and knowledge (static)
2. **User Context** 👤 - Personal profile (dynamic)
3. **Conversation Context** 💬 - Dialogue history (dynamic)
4. **Retrieved Context** 🔍 - Query-specific data (dynamic)

### Implementation Principles
- ✅ Build context **step-by-step** using simple data structures
- ✅ **Combine all four types** for intelligent responses
- ✅ **Adapt strategies** based on user type and conversation length
- ✅ **Balance richness with efficiency** to manage token limits

### What You Can Do Now
- 🔧 Build context-aware LLM applications
- 🎯 Personalize responses based on user profiles
- 💬 Maintain conversation flow with history
- 🔍 Integrate dynamic data retrieval

---

## 🚀 What's Next?

**Section 2: RAG Foundations**

You'll learn:
- 🔍 Advanced retrieval techniques with Redis
- 🎯 Vector similarity search
- 🏗️ Building production RAG systems with LangChain
- ⚡ Optimizing retrieval performance

**Continue to: `section-2-rag-foundations/01_building_your_rag_agent.ipynb` →**